In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import torch
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from collections import Counter
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

In [10]:
from Scripts.Configs.ConfigClass import Config

In [11]:
config = Config(r'C:\Users\fardin\Projects\ColorIntelligence\Scripts\Configs\Config.json')

In [2]:
# train_df = pd.read_csv(r'C:\Users\fardin\Projects\ColorIntelligence\data\Amazon-Review\train.csv')
# test_df = pd.read_csv(r'C:\Users\fardin\Projects\ColorIntelligence\data\Amazon-Review\test.csv')
# train_df[:50000].to_csv(r'C:\Users\fardin\Projects\ColorIntelligence\data\Amazon-Review\train_sm.csv', ',', index=False)
# test_df[:5000].to_csv(r'C:\Users\fardin\Projects\ColorIntelligence\data\Amazon-Review\test_sm.csv', ',', index=False)
train_df = pd.read_csv(r'C:\Users\fardin\Projects\ColorIntelligence\data\Amazon-Review\train_sm.csv')
test_df = pd.read_csv(r'C:\Users\fardin\Projects\ColorIntelligence\data\Amazon-Review\test_sm.csv')

In [3]:
train_df.columns = ['Polarity', 'Title', 'Review']
test_df.columns = ['Polarity', 'Title', 'Review']

In [4]:
train_df = train_df[['Polarity', 'Review']]
test_df = test_df[['Polarity', 'Review']]

In [ ]:
from torch_geometric.data import Data

In [5]:
train_df['Review'][0]

"I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."

In [30]:
# from Scripts.DataManager.GraphConstructor.CoOccurrenceGraphConstructor import CoOccurrenceGraphConstructor
import spacy
import numpy as np
from typing import Dict
import torch

In [49]:
nlp = spacy.load(config.spacy.pipeline)
doc = nlp(train_df['Review'][0])

In [98]:
unique_words = []
for token in doc:
    unique_words.append(token.lemma_)
unique_words = list(set(unique_words))
unique_map = pd.Series(range(len(unique_words)), index=unique_words, dtype=int)
n_gram = 4
token_lemmas = [t.lemma_ for t in doc]

In [99]:
unique_map['I']

47

In [108]:
dense_mat = torch.zeros((len(unique_words), len(unique_words)), dtype=torch.float32)
for i in range(len(token_lemmas)-n_gram):
    n_gram_data = set(token_lemmas[i:i+n_gram])
    n_gram_ids = [unique_map[val] for val in n_gram_data]
    grid_ids = torch.tensor([(x,y) for x in n_gram_ids for y in n_gram_ids])
    dense_mat[grid_ids[:,0], grid_ids[:,1]] += 1
dense_mat = torch.nn.functional.normalize(dense_mat)

In [109]:
sparse_mat = torch.sparse_coo_tensor(dense_mat)

TypeError: sparse_coo_tensor(): argument 'size' (position 1) must be tuple of ints, but found element of type Tensor at pos 0

In [104]:
dense_mat

tensor([[ 4.,  0.,  0.,  ...,  0.,  3.,  0.],
        [ 0.,  1.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  4.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  4.,  0.,  0.],
        [ 3.,  0.,  0.,  ...,  0., 12.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  4.]])

In [19]:



for sent in doc.sents:
    print(f'sent.text: {sent.text}')
    for token in sent:
        print(f'token.text: {token.lemma_}')


sent.text: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit.
token.text: I
token.text: be
token.text: read
token.text: a
token.text: lot
token.text: of
token.text: review
token.text: say
token.text: that
token.text: this
token.text: be
token.text: the
token.text: good
token.text: '
token.text: game
token.text: soundtrack
token.text: '
token.text: and
token.text: I
token.text: figure
token.text: that
token.text: I
token.text: would
token.text: write
token.text: a
token.text: review
token.text: to
token.text: disagree
token.text: a
token.text: bit
token.text: .
sent.text: This in my opinino is Yasunori Mitsuda's ultimate masterpiece.
token.text: this
token.text: in
token.text: my
token.text: opinino
token.text: be
token.text: Yasunori
token.text: Mitsuda
token.text: 's
token.text: ultimate
token.text: masterpiece
token.text: .
sent.text: The music is timeless and I'm been listening to it for years now and 

In [12]:

graph_constructor = CoOccurrenceGraphConstructor(train_df['Review'][0], config)

TypeError: tensor() missing 1 required positional arguments: "data"

In [ ]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(self.text)

self.x = torch.tensor()
self.y = torch.tensor()
self.edge_index = torch.tensor()